In [42]:
import numpy as np
from astropy.coordinates import SkyCoord, HeliocentricTrueEcliptic, ITRS, EarthLocation
from astropy.time import Time
import astropy.units as u
import torch as tn


tn.set_printoptions(precision=10, sci_mode=False)

# Time at CMS
time = Time.now()

# Define the location of CMS in terms of longitude, latitude and azimuth

azimuth = 1.7677    
latitude = 0.8082  
longitude = 0.1061   

# Distance from the Sun
distance_sun = 1 * u.au

# Define the Earth's angular velocity (rad/s)
omega_utc = 7.27221e-5     # Earth's angular velocity in rad/s at UTC.


omega_t = omega_utc*time.ut1.datetime.timestamp() + 3.2830    #omeag*t = omega_UTC * t_CMS +phi_UNIX +phi_longit

# Rotation matrices to go from ITRS to CMS frame

# rotation around the z-axis which makes the x-axis normal to the plane of the LHC.

R_z = tn.tensor([
        [1,0,0,0],
        [0,0,-1,0],
        [0,1,0,0],
        [0,0,0,1]
    ], dtype=tn.float32)


# To orient the z-axis towards the North. We rotate counterclockwise around the x′ axis with an angle π −θ (co-azimuth).
def R_x(angle):
    return tn.tensor([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, - np.cos(angle), np.sin(angle)],
        [0, 0, - np.sin(angle), - np.cos(angle)]
    ], dtype=tn.float32)


# Rotation around the y axis to align the z-axis with the Z-axis of the SCF.
def R_y(angle):
    return tn.tensor([
        [1, 0, 0, 0],
        [0, np.cos(angle), 0, np.sin(angle)],
        [0, 0, 1, 0],
        [0, -np.sin(angle), 0, np.cos(angle)]
    ], dtype=tn.float32)


# A final rotation around the Z-axis has two purposes: to follow the rotation of the Earth over time and to synchronize with the SCF:
def R_Z(angle):
    return tn.tensor([
        [1, 0 , 0, 0],
        [0, np.cos(angle), -np.sin(angle), 0],
        [0, np.sin(angle), np.cos(angle), 0],
        [0, 0, 0, 1]
    ], dtype=tn.float32)



# Construct the complete rotation matrix from SCF to CMS
R_matrix= R_y(latitude)@R_x(azimuth)@R_z@R_Z(omega_t) 

print(R_matrix)


tensor([[ 1.0000000000,  0.0000000000,  0.0000000000,  0.0000000000],
        [ 0.0000000000,  0.2692489624,  0.9526260495,  0.1414520144],
        [ 0.0000000000, -0.1694827080, -0.0977148414,  0.9806770086],
        [ 0.0000000000,  0.9480404854, -0.2880199552,  0.1351440251]])
